# Chapter 6

# Data Loading, Storage, and File Formats

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.read_csv('/Users/allenc/PyCharmProjects/JupyterProjects/WesWrangling/examples/ex1.csv')

pandas features a number of functions for reading tabular data as a DataFrame object. Table 6-1 summarizes some of them, though read_csv and read_table are likely the ones you’ll use the most.
I’ll give an overview of the mechanics of these functions, which are meant to convert text data into a DataFrame. The optional arguments for these functions may fall into a few categories:

Indexing
Can treat one or more columns as the returned DataFrame, and whether to get column names from the file, the user, or not at all.

Type inference and data conversion

This includes the user-defined value conversions and custom list of missing value markers.
Datetime parsing

Includes combining capability, including combining date and time information spread over multiple columns into a single column in the result.

Iterating
Support for iterating over chunks of very large files.

Unclean data issues
Skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

Here I used the Unix cat shell command to print the raw contents of the file to the screen. If you’re on Windows, you can use type instead of cat to achieve the same effect.

In [4]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [6]:
# Since this is comma-delimited, we can use read_csv to read it into a DataFrame:
df = pd.read_csv('examples/ex1.csv')

In [7]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [ ]:
We could also have used read_table and specified the delimiter:

In [8]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
# A file will not always have a header row. Consider this file:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

To read this file, you have a couple of options. You can allow pandas to assign default column names, or you can specify names yourself:

In [11]:
pd.read_csv('examples/ex2.csv', header = None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
pd.read_csv('examples/ex2.csv', names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the message column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named 'message' using the index_col argument:

In [14]:
names = ['a', 'b', 'c', 'd', 'message']

In [15]:
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [16]:
# In the event that you want to form a hierarchical index from multiple columns, pass a list of column numbers or names:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [18]:
parsed = pd.read_csv('examples/csv_mindex.csv',
                             index_col=['key1', 'key2'])

In [19]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. Consider a text file that looks like this:

In [20]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

While you could do some munging by hand, the fields here are separated by a vari‐ able amount of whitespace. In these cases, you can pass a regular expression as a delimiter for read_table. This can be expressed by the regular expression \s+, so we have then:

In [21]:
result = pd.read_table('examples/ex3.txt', sep='\s+')

In [22]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


The parser functions have many additional arguments to help you handle the wide variety of exception file formats that occur (see a partial listing in Table 6-2). For example, you can skip the first, third, and fourth rows of a file with skiprows:

In [23]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [24]:
pd.read_csv('examples/ex4.csv', skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file pars‐ ing process. Missing data is usually either not present (empty string) or marked by some sentinel value. By default, pandas uses a set of commonly occurring sentinels, such as NA and NULL:

In [25]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [26]:
result = pd.read_csv('examples/ex5.csv')

In [27]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [28]:
# The na_values option can take either a list or set of strings to consider missing values:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])

In [29]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [31]:
# Different NA sentinels can be specified for each column in a dict:
sentinels = {'message':['foo', 'NA'], 'something':['two']}

In [32]:
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


# Reading Text Files in Pieces

When processing very large files or figuring out the right set of arguments to cor‐ rectly process a large file, you may only want to read in a small piece of a file or iterate through smaller chunks of the file.
Before we look at a large file, we make the pandas display settings more compact:

In [35]:
pd.options.display.max_rows=10

In [37]:
result = pd.read_csv('examples/ex6.csv')

In [38]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [39]:
# If you want to only read a small number of rows (avoiding reading the entire file), specify that with nrows:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [42]:
# To read a file in pieces, specify a chunksize as a number of rows:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

In [43]:
chunker

The TextParser object returned by read_csv allows you to iterate over the parts of the file according to the chunksize. For example, we can iterate over ex6.csv, aggre‐ gating the value counts in the 'key' column like so:

In [46]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
tot = pd.Series([]) 
for piece in chunker:
        tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)

/var/folders/vm/hn4f87jj1kv93ht83n76p_d00000gp/T/ipykernel_3570/919391867.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


In [47]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

# Writing Data to Text Format

In [48]:
# Data can also be exported to a delimited format. Let’s consider one of the CSV files read before:
data = pd.read_csv('examples/ex5.csv')

In [49]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [50]:
# Using DataFrame’s to_csv method, we can write the data out to a comma-separated file:
data.to_csv('examples/out.csv')

In [51]:
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [52]:
# Other delimiters can be used, of course (writing to sys.stdout so it prints the text result to the console):
import sys

In [53]:
data.to_csv(sys.stdout, sep='!')

!something!a!b!c!d!message
0!one!1!2!3.0!4!
1!two!5!6!!8!world
2!three!9!10!11.0!12!foo


In [54]:
# Missing values appear as empty strings in the output. You might want to denote them by some other sentinel value:
data.to_csv(sys.stdout,na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [57]:
# With no other options specified, both the row and column labels are written. Both of these can be disabled:
data.to_csv(sys.stdout,index=False,header= False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [58]:
data.to_csv(sys.stdout, index=False, columns=['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [61]:
# Series also has a to_csv method:
dates = pd.date_range('1/1/2000', periods=7)

In [62]:
ts = pd.Series(np.arange(7), index=dates)

In [63]:
ts.to_csv('examples/tseries.csv')

In [64]:
!cat examples/tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


# Working with Delimited Formats

In [65]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


For any file with a single-character delimiter, you can use Python’s built-in csv mod‐ ule. To use it, pass any open file or file-like object to csv.reader:

In [66]:
import csv

In [67]:
f = open('examples/ex7.csv')

In [68]:
reader = csv.reader(f)

In [ ]:
Iterating through the reader like a file yields tuples of values with any quote charac‐
ters removed:

In [69]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


From there, it’s up to you to do the wrangling necessary to put the data in the form that you need it. Let’s take this step by step. First, we read the file into a list of lines:

In [70]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [71]:
# Then, we split the lines into the header line and the data lines:
header, values = lines[0], lines[1:]

Then we can create a dictionary of data columns using a dictionary comprehension
and the expression zip(*values), which transposes rows to columns:

In [72]:
data_dict = {h:v for h, v in zip(header, zip(*values))}

In [73]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV files come in many different flavors. To define a new format with a different delimiter, string quoting convention, or line terminator, we define a simple subclass of csv.Dialect:

In [75]:
class my_dialect(csv.Dialect): 
    lineterminator = '\n' 
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [76]:
reader = csv.reader(f, dialect=my_dialect)

ValueError: I/O operation on closed file.

In [78]:
# We can also give individual CSV dialect parameters as keywords to csv.reader without having to define a subclass:
reader = csv.reader(f, delimiter='|')

ValueError: I/O operation on closed file.

In [81]:
# To write delimited files manually, you can use csv.writer. It accepts an open, writa‐ ble file object and the same dialect and format options as csv.reader:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect) 
    writer.writerow(('one', 'two', 'three')) 
    writer.writerow(('1', '2', '3')) 
    writer.writerow(('4', '5', '6')) 
    writer.writerow(('7', '8', '9'))

# JSON Data

JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. Here is an example:

There are several Python libraries for reading and writing JSON data. I’ll use json here, as it is built into the Python standard library. To convert a JSON string to Python form, use json.loads:

In [84]:
 obj = """
    {"name": "Wes",
     "places_lived": ["United States", "Spain", "Germany"],
     "pet": null,
     "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
                  {"name": "Katie", "age": 38,
                   "pets": ["Sixes", "Stache", "Cisco"]}]
} """

In [85]:
import json

In [86]:
result = json.loads(obj)

In [87]:
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [88]:
# json.dumps, on the other hand, converts a Python object back to JSON:
asjson = json.dumps(result)

How you convert a JSON object or list of objects to a DataFrame or some other data structure for analysis will be up to you. Conveniently, you can pass a list of dicts (which were previously JSON objects) to the DataFrame constructor and select a sub‐ set of the data fields:

In [90]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])

In [91]:
siblings

,name,age
0,Scott,30
1,Katie,38


The pandas.read_json can automatically convert JSON datasets in specific arrange‐ ments into a Series or DataFrame. For example:

In [92]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [93]:
# The default options for pandas.read_json assume that each object in the JSON array is a row in the table:
data = pd.read_json('examples/example.json')

In [94]:
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [95]:
# If you need to export data from pandas to JSON, one way is to use the to_json meth‐ ods on Series and DataFrame:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [96]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


# XML and HTML: Web Scraping

In [98]:
# !pip3 install lxml
# !pip3 install beautifulsoup4 html5lib

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 2.2 MB/s eta 0:00:00a 0:00:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near fut

The pandas.read_html function has a number of options, but by default it searches for and attempts to parse all tabular data contained within <table> tags. The result is a list of DataFrame objects:

In [99]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [100]:
len(tables)

1

In [101]:
failures = tables[0]

In [102]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"
